# 1. Thư viện

In [1]:
import json
import re

from docx import Document

# 2. Thực thi

## 2.1. Tách lấy Chapters

### 2.1.1. Xử lý tách

In [2]:
def extract_chapters(docx_file):
    # Mở file docx
    doc = Document(docx_file)
    chapters = {}

    # Biến để lưu tên Chương và mô tả
    current_chapter = None
    current_description = None
    
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        # print(text)  # In nội dung đoạn văn để kiểm tra
        
        # Kiểm tra định dạng Chương (cả số La Mã và số Ả Rập)
        chapter_match = re.match(r'^(Chương\s+[IVXLCDM\d]+)\s*(.*)', text)
        if chapter_match:
            if current_chapter is not None and current_description is not None:
                # Lưu Chương trước đó vào dict
                chapters[current_chapter] = current_description
            
            # Cập nhật Chương hiện tại
            current_chapter = chapter_match.group(1)
            current_description = chapter_match.group(2).strip()
            continue
        
        # Nếu gặp đoạn văn chứa mô tả của Chương
        if current_chapter and not re.match(r'^(Mục|Điều)\s+\d+\.', text):
            current_description += ' ' + text.strip()
            continue

        # Kiểm tra nếu gặp Mục hoặc Điều
        if re.match(r'^(Mục|Điều)\s+\d+\.', text):
            # Nếu gặp Mục hoặc Điều, lưu lại Chương hiện tại
            if current_chapter is not None and current_description is not None:
                chapters[current_chapter] = current_description
                current_chapter = None
                current_description = None

    # Lưu Chương cuối cùng nếu có
    if current_chapter is not None and current_description is not None:
        chapters[current_chapter] = current_description

    print(f'Found {len(chapters)} chapters')  # In ra số lượng Chương tìm thấy
    return chapters

### 2.1.2. Lưu ra .json

In [3]:
def save_to_json(data, json_file):
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

### 2.1.3. Thực thi

In [4]:
# Sử dụng hàm
docx_file = r'data\trich_dan_luat\docx\Luật_Quốc hội_10-2022-QH15_Nội vụ_Chính quyền địa phương_10-11-2022_01-07-2023_thực hiện dân chủ ở cơ sở.docx'  # Thay thế bằng đường dẫn đến file .docx của bạn
json_file = r'data\trich_dan_luat\json\chapters.json'  # Tên file .json bạn muốn lưu

chapters = extract_chapters(docx_file)
save_to_json(chapters, json_file)
print(f'Data has been saved to {json_file}')

Found 6 chapters
Data has been saved to data\trich_dan_luat\json\chapters.json


## 2.2. Tách lấy Sections

### 2.2.1. Xử lý tách

In [5]:
def extract_sections(docx_file):
    # Mở file docx
    doc = Document(docx_file)
    sections = {}

    # Biến để lưu tên Mục và mô tả
    current_section = None
    current_description = None
    
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        
        # Kiểm tra định dạng Mục
        section_match = re.match(r'^(Mục\s+\d+)\.\s*(.*)', text)
        if section_match:
            if current_section is not None and current_description is not None:
                # Lưu Mục trước đó vào dict
                sections[current_section] = current_description.strip()
            
            # Cập nhật Mục hiện tại
            current_section = section_match.group(1)
            current_description = section_match.group(2).strip()
            continue
        
        # Nếu gặp Điều thì dừng tích lũy mô tả của Mục
        if re.match(r'^Điều\s+\d+\.', text):
            if current_section is not None and current_description is not None:
                sections[current_section] = current_description.strip()
                current_section = None
                current_description = None
            continue
        
        # Tích lũy mô tả của Mục nếu không gặp Điều hoặc Chương
        if current_section and not re.match(r'^(Chương|Điều)\s+[IVXLCDM\d]+', text):
            current_description += ' ' + text.strip()
    
    # Lưu Mục cuối cùng nếu có
    if current_section is not None and current_description is not None:
        sections[current_section] = current_description.strip()

    print(f'Found {len(sections)} sections')  # In ra số lượng Mục tìm thấy
    return sections

### 2.2.2. Lưu thành .json

In [6]:
def save_to_json(data, json_file):
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

### 2.2.3. Thực thi

In [7]:
# Sử dụng hàm
docx_file = r'data\trich_dan_luat\docx\Luật_Quốc hội_10-2022-QH15_Nội vụ_Chính quyền địa phương_10-11-2022_01-07-2023_thực hiện dân chủ ở cơ sở.docx'  # Thay thế bằng đường dẫn đến file .docx của bạn
json_file = r'data\trich_dan_luat\json\sections.json'  # Tên file .json bạn muốn lưu

sections = extract_sections(docx_file)
save_to_json(sections, json_file)
print(f'Data has been saved to {json_file}')

Found 4 sections
Data has been saved to data\trich_dan_luat\json\sections.json
